# End-to-End Example

We will be looking at Radio Data System (RDS), which is a communications protocol for embedding small amounts of information in FM radio broadcasts, such as station and song name. We will have to demodulate FM, frequency shift, filter, decimate, resample, synchronize, decode, and parse the bytes. An example IQ file is provided for testing purposes or if you don’t have an SDR handy.

## Introduction to FM Radio and RDS

To understand RDS we must first review FM radio broadcasts and how their signals are structured. You are probably familiar with the audio portion of FM signals, which are simply audio signals frequency modulated and transmitted at center frequencies corresponding to the station’s name, e.g., “WPGC 95.5 FM” is centered at exactly 95.5 MHz. In addition to the audio portion, each FM broadcast contains some other components that are frequency modulated along with the audio. Instead of just Googling the signal structure, let’s take a look at the power spectral density (PSD) of an example FM signal, after the FM demodulation. We only view the positive portion because the output of FM demodulation is a real signal, even though the input was complex (we will view the code to perform this demodulation shortly).